In [1]:
!pip install pandas transformers torch tqdm

In [2]:
import pandas as pd

# kalau file kamu di-upload ke Colab manual:
from google.colab import files
uploaded = files.upload()

# setelah upload, misal nama file kamu 'reviews.csv'
df = pd.read_csv("reviews.csv")
df.head()


Saving reviews.csv to reviews.csv


,review_id,product_id,date,rating,sentiment,platform,comment
0,R100000,PC014,10/16/2024,4.6,Positive,Instagram,"Packaging bocor saat diterima, kurang aman."
1,R100001,PC012,6/27/2023,2.8,Negative,Instagram,"Mudah dibeli saat promo, value for money."
2,R100002,PC003,10/21/2022,3.2,Neutral,Shopee,"Mudah dibeli saat promo, value for money."
3,R100003,PC010,3/7/2025,2.0,Negative,Tokopedia,"Mudah dibeli saat promo, value for money."
4,R100004,PC015,3/3/2022,1.5,Negative,Shopee,Memberikan hasil sesuai klaim after 2 weeks.


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "taufiqdp/indonesian-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/922 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
from tqdm import tqdm

tqdm.pandas()

# apply NLP model ke setiap baris comment
df["predicted_sentiment"] = df["comment"].progress_apply(lambda x: sentiment_pipeline(x)[0]["label"])
df.head()


100%|██████████| 10000/10000 [20:31<00:00,  8.12it/s]


,review_id,product_id,date,rating,sentiment,platform,comment,predicted_sentiment
0,R100000,PC014,10/16/2024,4.6,Positive,Instagram,"Packaging bocor saat diterima, kurang aman.",negatif
1,R100001,PC012,6/27/2023,2.8,Negative,Instagram,"Mudah dibeli saat promo, value for money.",netral
2,R100002,PC003,10/21/2022,3.2,Neutral,Shopee,"Mudah dibeli saat promo, value for money.",netral
3,R100003,PC010,3/7/2025,2.0,Negative,Tokopedia,"Mudah dibeli saat promo, value for money.",netral
4,R100004,PC015,3/3/2022,1.5,Negative,Shopee,Memberikan hasil sesuai klaim after 2 weeks.,netral


In [5]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1WDeDjtv18DB_zuw4FSV80Ba3sIqAbNGIFWHUir-Gi9A/edit#gid=0


In [6]:
df = df.drop(columns=["consistency"])

KeyError: "['consistency'] not found in axis"

In [7]:
df = df.drop(columns=["rating_label"])

KeyError: "['rating_label'] not found in axis"

In [8]:
# Konversi rating (angka 1–5) menjadi label sentimen
def label_from_rating(r):
    if r >= 4.0:
        return "positif"
    elif r >= 3:
        return "netral"
    else:
        return "negatif"

# Terapkan fungsi ke kolom rating
df["rating_label"] = df["rating"].apply(label_from_rating)

# Cek hasil konversi awal
df[["rating", "rating_label"]].head(10)


,rating,rating_label
0,4.6,positif
1,2.8,negatif
2,3.2,netral
3,2.0,negatif
4,1.5,negatif
5,4.9,positif
6,3.1,netral
7,1.5,negatif
8,4.2,positif
9,2.0,negatif


In [9]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1Xt466IgG0ctcLPzVLvUDD8ncz6CvAu2VfXDs-u4_H2Y/edit#gid=0


In [11]:
# Bandingkan hanya antara hasil prediksi model NLP dan label dari rating
def check_consistency(row):
    if row["predicted_sentiment"] == row["rating_label"]:
        return "✅ Consistent"
    else:
        return "❌ Inconsistent"

# Tambahkan kolom hasil ke dataframe utama
df["consistency"] = df.apply(check_consistency, axis=1)

# Cek jumlah hasil awal
df["consistency"].value_counts()

,count
consistency,
❌ Inconsistent,6529
✅ Consistent,3471
